In [ ]:
!pip install kaggle

In [ ]:
# For Google Colab only
from google.colab import files
files.upload()  # Upload kaggle.json here

In [ ]:
# kaggle key activation

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Data retrieval
!kaggle datasets download -d sainikhileshreddy/food-recognition-2022
!unzip -q food-recognition-2022.zip -d dataset

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

def show_images(images, titles=None, ncols=3):
    n = len(images)
    nrows = n // ncols + (1 if n % ncols else 0)
    fig, axs = plt.subplots(nrows, ncols, figsize=(15, 5 * nrows))
    axs = axs.flatten()
    for i, img_path in enumerate(images):
        img = Image.open(img_path)
        axs[i].imshow(img)
        axs[i].axis('off')
        if titles:
            axs[i].set_title(titles[i])
    plt.tight_layout()
    plt.show()

# Show first few images from a specific food category
food_category = './dataset/raw_data/public_training_set_release_2.0/images'  # Change as needed
image_files = [os.path.join(food_category, f) for f in os.listdir(food_category)[:9]]
show_images(image_files)

In [ ]:
import tensorflow as tf
import json
from PIL import Image
import numpy as np


def create_dataset(image_dir, annotations_file, batch_size, img_height, img_width):
    # Read and parse the JSON file
    with open(annotations_file, 'r') as file:
        data = json.load(file)

    # Create a mapping from ID to all metadata
    id_to_metadata = {item['id']: item for item in data['categories']}

    # Initialize lists to store file paths and metadata
    file_paths = []
    metadata_list = []

    # Iterate through the directory and match each image with its metadata
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_id = int(filename.split('.')[0])
            image_metadata = id_to_metadata.get(image_id)
            if image_metadata:
                file_paths.append(os.path.join(image_dir, filename))
                metadata_list.append(image_metadata)

    # Load images and preprocess them
    def load_and_preprocess_image(path, metadata):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [img_height, img_width])
        image /= 255.0  # Normalize to [0,1] range

        # Prepare the metadata. Here we are converting it to a tensor.
        # You might need to adjust this depending on how you plan to use the metadata.
        metadata_tensor = tf.convert_to_tensor([metadata['id'],
                                               metadata['name'],
                                               metadata['name_readable'],
                                               metadata['supercategory']], dtype=tf.string)
        return image, metadata_tensor

    # Create a dataset
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, metadata_list))
    dataset = dataset.map(load_and_preprocess_image)
    dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
# Define image size and batch size
img_height, img_width = 224, 224  # You can adjust these values as per your model's requirement
batch_size = 32

# Rescaling factor for normalization (1./255 is common for RGB images)
rescale_factor = 1./255

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

num_classes = 11 # num_classes is the number of food categories

sequential_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])